In [2]:
import numpy as np
import torch as th
import gym
import gym.spaces as spaces

from Env.env import BSMarket

In [3]:
env = BSMarket(n_assets=50000,
               cost=0.02,
                 n_periods=30,
                 freq=1,
                 period_unit=365,
                 drift=0.0,
                 volatility=0.2,
                 init_price=1.0,
                 risk_free_interest=0.0,
                 strike=1.0,
                 dividend=0.0,
                 payoff="european",
                 gen_name="gbm",
                 reward_mode="pnl")